In [1]:
%cd ../

/home/qwj/code/HippoRAG


## utils

In [3]:
import os
import shutil

def is_new_files(file_name):
    return not (file_name.startswith("openie_") or file_name.startswith('.') or file_name.endswith("named_entity_output.tsv"))

def move_files(subfolder_name):
    # 定义新子文件夹的名称
    new_subfolder = os.path.join('output', subfolder_name)
    
    # 如果新子文件夹不存在，创建它
    if not os.path.exists(new_subfolder):
        os.makedirs(new_subfolder)

    # 获取文件夹下的所有文件（不包含子文件夹）
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
        
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                # 移动文件到新子文件夹
                print(f"Moved: {file_name}")
                shutil.move(file_path, os.path.join(new_subfolder, file_name))

def show_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
    
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(file_name)


In [4]:
import json
import pickle

read_json = lambda path: json.load(open(path, 'r'))
read_pickle = lambda path: pickle.load(open(path, 'rb'))

In [5]:
import itertools
from typing import TypedDict, List, Dict, NamedTuple, Tuple

## Musique + GPT + COLBERT

### extract NER

In [4]:
!python src/create_graph.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --extraction_type ner 

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10316.67it/s]
Correct Wiki Format: 0 out of 11656
100%|████████████████████████████████████| 1006/1006 [00:00<00:00, 71231.51it/s]


In [5]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
kb_to_kb.tsv


### compute similar edges

#### kb internal entity similarity

In [7]:
!CUDA_VISIBLE_DEVICES="2,3" python src/colbertv2_knn.py --filename output/kb_to_kb.tsv



[Jul 01, 07:42:44] #> Creating directory colbert/indexes/nbits_2 


#> Starting...
nranks = 1 	 num_gpus = 2 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "exp\/colbert

#### query to kb similarity

In [8]:
!CUDA_VISIBLE_DEVICES="2,3" python src/colbertv2_knn.py --filename output/query_to_kb.tsv



[Jul 01, 07:53:18] #> Note: Output directory colbert/indexes/nbits_2 already exists


[Jul 01, 07:53:18] #> Will delete 22 files already at colbert/indexes/nbits_2 in 20 seconds...
#> Starting...
nranks = 1 	 num_gpus = 2 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction"

In [9]:
!ls data/lm_vectors/colbert

corpus.tsv		     nearest_neighbor_query_to_kb.p
nearest_neighbor_kb_to_kb.p  queries.tsv


In [16]:
kb_to_kb_sim: Dict[str, Tuple[List[str], List[float]]] = read_pickle("data/lm_vectors/colbert/nearest_neighbor_kb_to_kb.p")
len(kb_to_kb_sim)

91729

In [18]:
query_to_kb_sim: Dict[str, Tuple[List[str], List[float]]] = read_pickle("data/lm_vectors/colbert/nearest_neighbor_query_to_kb.p")
len(query_to_kb_sim)

1234

### `create_graph`


In [17]:
!python src/create_graph.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --extraction_type ner \
    --threshold 0.8 \
    --create_graph \
    --cosine_sim_edges

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10829.59it/s]
Correct Wiki Format: 0 out of 11656
100%|████████████████████████████████████| 1006/1006 [00:00<00:00, 69863.40it/s]
Creating Graph
100%|██████████████████████████████████| 11656/11656 [00:00<00:00, 11829.07it/s]
Loading Vectors
Augmenting Graph from Similarity
100%|██████████████████████████████████| 91729/91729 [00:01<00:00, 66869.42it/s]
Saving Graph
                                                         1
0                                                         
Total Phrases                                       327435
Unique Phrases                                       91729
Number of Individual Triples                        109145
Number of Incorrectly Formatted Triples (ChatGP...    1136
Number of Triples w/o NER Entities (ChatGPT Error)    6762
Number of Unique Individual Triples                 107448
Number of Entities                                  218290
Number of Relations             

move files

In [21]:
show_new_files()

musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p
musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
query_to_kb.tsv
rel_kb_to_kb.tsv
musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
kb_to_kb.tsv
musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
musique_facts_and_sim_graph_doc_to_facts_csr_ents_only_lower_preprocess_ner.v3.subset.p
musique_facts_and_sim_graph_facts_to_phrases_ents_only_l

In [22]:
move_files('musique_gpt')

Moved: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p
Moved: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: query_to_kb.tsv
Moved: rel_kb_to_kb.tsv
Moved: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Moved: musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Moved: kb_to_kb.tsv
Moved: musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
Moved: musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
Moved: musique_facts_and_sim_graph_doc_to_facts_csr_ents_onl

## look at files

In [7]:
base_dir = "./output/musique_gpt"

### triplet list

In [8]:
class TripletINFO(TypedDict):
    idx: int
    head: str
    relation: str
    tail: str

triplet_list: List[TripletINFO] = read_json(base_dir+"/musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json")
print(len(triplet_list))
print(triplet_list[:5])

109145
[{'idx': 0, 'head': 'lionel messi', 'relation': 'enrolled in', 'tail': 'royal spanish football federation'}, {'idx': 1, 'head': 'lionel messi', 'relation': 'befriended', 'tail': 'cesc f bregas'}, {'idx': 2, 'head': 'lionel messi', 'relation': 'befriended', 'tail': 'gerard piqu'}, {'idx': 3, 'head': 'lionel messi', 'relation': 'part of', 'tail': 'baby dream team'}, {'idx': 4, 'head': 'lionel messi', 'relation': 'played for', 'tail': 'barcelona'}]


### fact dict

In [9]:
class Triplet(NamedTuple):
    head: str
    relation: str
    tail: str

raw_triplet_to_idx = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p")
triplet_to_idx = {
    Triplet(head=k[0], relation=k[1], tail=k[2]): v for k, v in raw_triplet_to_idx.items()
}
len(triplet_to_idx)

107448

In [15]:
idx_to_triplet = {v:k for k, v in triplet_to_idx.items()}

In [19]:
triplet_list_2 = [Triplet(head=t['head'], relation=t['relation'], tail=t['tail']) for t in triplet_list]
len(triplet_list_2), len(set(triplet_list_2))

(109145, 107448)

### entity list

In [25]:
class EntityINFO(TypedDict):
    idx: str
    name: str

entity_list: List[EntityINFO] = read_json(base_dir+"/musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json")
print(len(entity_list))
print(entity_list[:5])

91729
[{'idx': 0, 'name': ''}, {'idx': 1, 'name': '0'}, {'idx': 2, 'name': '0    0'}, {'idx': 3, 'name': '0    2'}, {'idx': 4, 'name': '0   1 972'}]


### similarity

In [32]:
class SimilarEntities(NamedTuple):
    name: str
    sim_score: float

similar_entities: List[Tuple[str, List[SimilarEntities]]] = read_pickle(f"{base_dir}/musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p")
print(len(similar_entities))

91729


### graph

In [37]:
graph: Dict[str, Dict[str, Tuple[str, float]]] = read_json(f"{base_dir}/musique_facts_and_sim_graph_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json")
len(graph)

91729

In [46]:
for key, value in itertools.islice(graph.items(),3):
    print(key, value)
    break

lionel messi {'royal spanish football federation': ['triple', 1], 'cesc f bregas': ['triple', 1], 'gerard piqu': ['triple', 1], 'baby dream team': ['triple', 1], 'barcelona': ['triple', 5], 'cadetes a': ['triple', 1], 'league': ['triple', 1], 'spanish cup': ['triple', 1], 'catalan cup': ['triple', 1], 'arsenal': ['triple', 1], 'fc barcelona': ['triple', 1], 'top scorer': ['triple', 1], '46 goals in the league': ['triple', 1], 'two hat tricks': ['triple', 1], 'barcelona s 5    4 victory over sevilla': ['triple', 1], 'defeat against athletic bilbao': ['triple', 1], 'uefa champions league': ['triple', 1], 'barcelona s match against las palmas': ['triple', 1], 'barcelona s 4    0 away win over real madrid in el cl sico': ['triple', 1], '2015 fifa club world cup final': ['triple', 1], 'silver ball': ['triple', 1], 'barcelona in a 4    0 home win over real betis': ['triple', 1], 'rosario  argentina': ['triple', 1], 'fifa u 20 world cup': ['triple', 1], 'copa am rica': ['triple', 2], 'olympic

### docs

In [36]:
# musique_facts_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
class PassageINFO(TypedDict):
    title: str
    text: str
    passage: str
    extracted_entities: List[str]
    extracted_triples: List[tuple]
    entities: List[tuple]
    clean_triples: List[tuple]
    noisy_triples: List[tuple]

graph_passage: List[PassageINFO] = read_json(f"{base_dir}/musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json")
print(len(graph_passage))

11656


### Entity Index

In [20]:
# musique_facts_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
entity_to_id: Dict[str, int] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p")
len(entity_to_id)

91729

### doc edges

In [49]:
doc_edges: Dict[Tuple[int, int], float] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p")
len(doc_edges)

206863

### doc_to_facts

In [50]:
## dict
doc_to_facts: Dict[Tuple[int, int], float] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p")
len(doc_to_facts)

108914

In [52]:
set(list(doc_to_facts.values()))

{1}

In [55]:
from scipy.sparse import csr_array


In [54]:
doc_to_fact_matrix: csr_array = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_doc_to_facts_csr_ents_only_lower_preprocess_ner.v3.subset.p")

doc_to_fact_matrix.shape


(11656, 109145)

### facts to phrases

In [56]:
## dict
facts_to_phrases: Dict[Tuple[int, int], float] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_facts_to_phrases_ents_only_lower_preprocess_ner.v3.subset.p")
len(facts_to_phrases)

214783

In [58]:
facts_to_phrases_matrix: csr_array = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_facts_to_phrases_csr_ents_only_lower_preprocess_ner.v3.subset.p")

facts_to_phrases_matrix.shape


(109145, 91729)

### entity_graph

In [62]:
entity_graph: Dict[Tuple[int, int], float] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_mean_0.8_thresh_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p")
len(entity_graph)

394781

In [65]:
min(entity_graph.values()), max(entity_graph.values())

(0.8, 11.0)

In [66]:
start_idx = [k[0] for k in entity_graph.keys()]
end_idx = [k[1] for k in entity_graph.keys()]

max(start_idx), max(end_idx)

(91728, 91728)

### relation_dict

In [31]:
relation_dict: Dict[Tuple[str, str], str] = read_pickle(f"{base_dir}/musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p")
len(relation_dict)

294610

In [26]:
list(relation_dict.keys())[:10]

[('lionel messi', 'royal spanish football federation'),
 ('lionel messi', 'cesc f bregas'),
 ('lionel messi', 'gerard piqu'),
 ('lionel messi', 'baby dream team'),
 ('lionel messi', 'barcelona'),
 ('lionel messi', 'cadetes a'),
 ('lionel messi', 'league'),
 ('lionel messi', 'spanish cup'),
 ('lionel messi', 'catalan cup'),
 ('lionel messi', 'arsenal')]

In [32]:
relation_dict_cp  = relation_dict.copy()
for t in triplet_list:
    edge = (t['head'], t['tail'])
    if edge in relation_dict_cp:
        relation_dict_cp.pop(edge)
    edge = (t['tail'], t['head'])
    if edge in relation_dict_cp:
        relation_dict_cp.pop(edge)
len(relation_dict_cp)

187918

In [33]:
set(relation_dict_cp.values())

{'equivalent'}

## New KG class

In [3]:
import sys
sys.path.append('./src')
from kb_utils import ExperimentConfig, KnowledgeBase

config = ExperimentConfig(
    dataset='musique',
    graph_type='facts_and_sim',
    retrieval_model_name='colbertv2',
    extraction_model_name='gpt-3.5-turbo-1106',
    base_dir='./output/musique_gpt'
)
kb = KnowledgeBase.build_from_config(config)
print(kb)

building knowledge graph: 100%|██████████| 298594/298594 [00:01<00:00, 149850.18it/s]

KnowledgeBase(91729 entities, 22222 relations, 298594 triplets)


In [6]:
kb.kb[64165]

Entity(name=pam, 31 relations)

In [7]:
kb.kb[64165].relations

{TargetEntity(-- breastfeeding -> cecelia),
 TargetEntity(-- equivalent -> jim and pam),
 TargetEntity(-- equivalent -> pam beesly),
 TargetEntity(-- equivalent -> pam belluck),
 TargetEntity(-- equivalent -> pam faris),
 TargetEntity(-- equivalent -> pam for dinner),
 TargetEntity(-- equivalent -> pam grier),
 TargetEntity(-- equivalent -> pam martin),
 TargetEntity(-- equivalent -> pam s drawing),
 TargetEntity(-- equivalent -> pam s graphic design projects),
 TargetEntity(-- equivalent -> pam s mother),
 TargetEntity(-- equivalent -> pam s voicemail messages),
 TargetEntity(-- equivalent -> pam s voicemails),
 TargetEntity(-- equivalent -> pam s work),
 TargetEntity(-- equivalent -> pam shriver),
 TargetEntity(-- equivalent -> pam slapping michael),
 TargetEntity(-- equivalent -> pam tillis),
 TargetEntity(-- equivalent -> pamal broadcasting),
 TargetEntity(-- equivalent -> pambula river),
 TargetEntity(-- is -> let know by jim that he is seeing someone),
 TargetEntity(-- is -> over